In [1]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

# to prevent kernel death because of

# 2019-03-30 17:11:04.317013: I tensorflow/core/common_runtime/process_util.cc:69] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
# OMP: Error #15: Initializing libiomp5.dylib, but found libiomp5.dylib already initialized.
# OMP: Hint: This means that multiple copies of the OpenMP runtime have been linked into the program. That is dangerous, since it can degrade performance or cause incorrect results. The best thing to do is to ensure that only a single OpenMP runtime is linked into the process, e.g. by avoiding static linking of the OpenMP runtime in any library. As an unsafe, unsupported, undocumented workaround you can set the environment variable KMP_DUPLICATE_LIB_OK=TRUE to allow the program to continue to execute, but that may cause crashes or silently produce incorrect results. For more information, please see http://www.intel.com/software/products/support/.

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
SMALL_DATASET_DIR = '../../nmnist/notMNIST_small/'
LARGE_DATASET_DIR = '../../nmnist/notMNIST_large/'
CACHE = {}
LABEL_MAP = {}
INV_LABEL_MAP = {}

In [3]:
def read(data_dir, override=False):
    f_v = 0
    global CACHE
    if not CACHE.get(data_dir, []) or override:
        CACHE[data_dir] = []
        X, y = [], []
        for f in tqdm(os.listdir(data_dir), desc='Letter'):
            if not f.startswith('.'):
                img_dir = os.path.join(data_dir, f)
                for img in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, img)
                    data = cv2.imread(img_path, 0)
                    if data is None:
                        continue
                    X.append(data * 2 / 255 - 1)
                    if LABEL_MAP.get(f) is None:
                        LABEL_MAP[f] = f_v
                        INV_LABEL_MAP[f_v] = f
                        f_v += 1
                    y.append(LABEL_MAP[f])
        CACHE[data_dir].append(np.array(X))
        CACHE[data_dir].append(np.array(y))
    return CACHE[data_dir][0], CACHE[data_dir][1]

In [4]:
def get_data(data_dir, verbose=False, override=False):
    X, y = read(data_dir, override=override)
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    if verbose:
        print(X.shape)
        print(y.shape)
        print(X[:5])
        print(y[:5])
        print(np.unique(y))
    return X, y

In [5]:
def get_split_data(data_dir, size=(0.7, 0.3), verbose=False, random_state=6, override=False):
    X, y = shuffle(*get_data(data_dir, verbose=verbose, override=override), random_state=random_state)
    assert abs(np.sum(size) - 1.0) < 0.001
    if verbose:
        print('splitting...')
#     X = X[:60000]
#     y = y[:60000]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [6]:
X_train, y_train, X_test, y_test = get_split_data(LARGE_DATASET_DIR, override=False, verbose=True)
print('reshaping...')
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)


(529114, 28, 28)
(529114,)
[[[-0.12156863 -0.83529412 -0.59215686 ... -1.         -1.
   -1.        ]
  [-0.11372549 -0.38823529 -0.71764706 ... -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         ... -1.         -1.
   -1.        ]
  ...
  [-1.         -1.         -1.         ... -1.         -1.
   -1.        ]
  [-1.         -1.         -1.         ... -0.98431373 -0.99215686
   -1.        ]
  [-1.         -1.         -1.         ... -1.         -1.
   -1.        ]]

 [[-1.         -1.         -0.96862745 ... -0.05882353 -0.16862745
   -0.3254902 ]
  [-1.         -1.         -1.         ... -0.01960784 -0.00392157
   -0.77254902]
  [-1.         -0.92941176 -0.12941176 ... -0.01176471 -0.0745098
   -0.52941176]
  ...
  [-1.         -0.75686275 -0.12941176 ... -0.97647059 -1.
   -1.        ]
  [-1.         -0.15294118 -0.00392157 ... -0.97647059 -1.
   -1.        ]
  [-1.         -0.27843137 -0.15294118 ... -1.         -1.
   -1.        ]]

 [[-1.         -1.       

$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x11x8 \rightarrow flatten \rightarrow 968 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [8]:
lr = 0.1
epochs = 10
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Learning rate: 0.1
Epochs: 10

Epoch 1/10
370379/370379 [==============================] - 142s 384us/step - loss: 0.4329 - acc: 0.8696
Epoch 2/10
370379/370379 [==============================] - 149s 402us/step - loss: 0.3275 - acc: 0.9006
Epoch 3/10
370379/370379 [==============================] - 138s 374us/step - loss: 0.2936 - acc: 0.9103
Epoch 4/10
370379/370379 [==============================] - 136s 367us/step - loss: 0.2721 - acc: 0.9165
Epoch 5/10
370379/370379 [==============================] - 150s 405us/step - loss: 0.2551 - acc: 0.9210
Epoch 6/10
370379/370379 [==============================] - 149s 402us/step - loss: 0.2391 - acc: 0.9257
Epoch 7/10
370379/370379 [==============================] - 146s 394us/step - loss: 0.2269 - acc: 0.9291
Epoch 8/10
370379/370379 [==============================] - 142s 384us/step - loss: 0.2162 - acc: 0.9321
Epoch 9/10
370379/370379 [==============================] - 138s 371us/step - loss: 0.2056 - acc: 0.9352
Epoch 10/10
158735/15873

Vertical:
$$28x28 \rightarrow (conv: f=(5, 3), s=1, c=6) \rightarrow 24x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x12x8 \rightarrow flatten \rightarrow 1056 \rightarrow FC \rightarrow 256 \rightarrow FC \rightarrow 64 \rightarrow softmax$$

In [9]:
lr = 0.1
epochs = 10
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 10

Epoch 1/10
370379/370379 [==============================] - 161s 435us/step - loss: 0.4502 - acc: 0.8607
Epoch 2/10
370379/370379 [==============================] - 155s 419us/step - loss: 0.3246 - acc: 0.8989
Epoch 3/10
370379/370379 [==============================] - 164s 442us/step - loss: 0.2868 - acc: 0.9102
Epoch 4/10
370379/370379 [==============================] - 164s 442us/step - loss: 0.2598 - acc: 0.9181
Epoch 5/10
370379/370379 [==============================] - 162s 438us/step - loss: 0.2402 - acc: 0.9234
Epoch 6/10
370379/370379 [==============================] - 167s 450us/step - loss: 0.2241 - acc: 0.9282
Epoch 7/10
370379/370379 [==============================] - 155s 419us/step - loss: 0.2091 - acc: 0.9328
Epoch 8/10
370379/370379 [==============================] - 153s 412us/step - loss: 0.1984 - acc: 0.9355
Epoch 9/10
370379/370379 [==============================] - 146s 395us/step - loss: 0.1887 - acc: 0.9384
Epoc

LeNet-5 + vertical:
$$
28x28 \rightarrow (conv: f=(5, 3), s=1, c=6, tanh) \rightarrow 24x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x13x6 \rightarrow (conv: f=3, s=1, c=8, tanh) \rightarrow 10x11x8 \rightarrow (maxpool: f=2, s=2) \rightarrow 5x5x8 \rightarrow flatten \rightarrow 200 \rightarrow FC(relu) \rightarrow 120 \rightarrow FC(relu) \rightarrow 84 \rightarrow softmax
$$

In [10]:
lr = 0.1
epochs = 10
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(84, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 10

Epoch 1/10
370379/370379 [==============================] - 220s 594us/step - loss: 0.4531 - acc: 0.8614
Epoch 2/10
370379/370379 [==============================] - 224s 605us/step - loss: 0.3525 - acc: 0.8909
Epoch 3/10
370379/370379 [==============================] - 223s 602us/step - loss: 0.3263 - acc: 0.8984
Epoch 4/10
370379/370379 [==============================] - 216s 583us/step - loss: 0.3111 - acc: 0.9027
Epoch 5/10
370379/370379 [==============================] - 212s 571us/step - loss: 0.3000 - acc: 0.9060
Epoch 6/10
370379/370379 [==============================] - 218s 587us/step - loss: 0.2922 - acc: 0.9085
Epoch 7/10
370379/370379 [==============================] - 216s 582us/step - loss: 0.2860 - acc: 0.9100
Epoch 8/10
370379/370379 [==============================] - 204s 550us/step - loss: 0.2812 - acc: 0.9114
Epoch 9/10
370379/370379 [==============================] - 199s 537us/step - loss: 0.2767 - acc: 0.9128
Epoc

Custom with maxpool:
$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 13x13x6 \rightarrow (conv: f=3, s=2, c=16) \rightarrow 5x5x16 \rightarrow (maxpool: f=2, s=1) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 258 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [12]:
lr = 0.1
epochs = 10
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=(2, 2), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 10

Epoch 1/10
370379/370379 [==============================] - 145s 390us/step - loss: 0.4383 - acc: 0.8676
Epoch 2/10
370379/370379 [==============================] - 143s 387us/step - loss: 0.3475 - acc: 0.8944
Epoch 3/10
370379/370379 [==============================] - 143s 386us/step - loss: 0.3229 - acc: 0.9013
Epoch 4/10
370379/370379 [==============================] - 143s 386us/step - loss: 0.3081 - acc: 0.9052
Epoch 5/10
370379/370379 [==============================] - 144s 389us/step - loss: 0.2987 - acc: 0.9081
Epoch 6/10
370379/370379 [==============================] - 144s 388us/step - loss: 0.2914 - acc: 0.9106
Epoch 7/10
370379/370379 [==============================] - 149s 402us/step - loss: 0.2848 - acc: 0.9121
Epoch 8/10
370379/370379 [==============================] - 146s 395us/step - loss: 0.2795 - acc: 0.9139
Epoch 9/10
370379/370379 [==============================] - 144s 390us/step - loss: 0.2748 - acc: 0.9151
Epoc

LeNet-5:
$$
28x28 \rightarrow (conv: f=5, s=1, c=6, tanh) \rightarrow 24x24x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x12x6 \rightarrow (conv: f=5, s=1, c=16) \rightarrow 8x8x16 \rightarrow (maxpool: f=2, s=2) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 256 \rightarrow FC \rightarrow 128 \rightarrow FC \rightarrow 64 \rightarrow softmax
$$

In [13]:
lr = 0.1
epochs = 10
print('Building LeNet-5...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='tanh'),
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building LeNet-5...
Learning rate: 0.1
Epochs: 10

Epoch 1/10
370379/370379 [==============================] - 223s 603us/step - loss: 0.4256 - acc: 0.8720
Epoch 2/10
370379/370379 [==============================] - 220s 593us/step - loss: 0.3434 - acc: 0.8944
Epoch 3/10
370379/370379 [==============================] - 220s 594us/step - loss: 0.3212 - acc: 0.9006
Epoch 4/10
370379/370379 [==============================] - 221s 597us/step - loss: 0.3077 - acc: 0.9050
Epoch 5/10
370379/370379 [==============================] - 220s 595us/step - loss: 0.2990 - acc: 0.9070
Epoch 6/10
370379/370379 [==============================] - 223s 603us/step - loss: 0.2907 - acc: 0.9097
Epoch 7/10
370379/370379 [==============================] - 231s 623us/step - loss: 0.2850 - acc: 0.9108
Epoch 8/10
370379/370379 [==============================] - 231s 623us/step - loss: 0.2805 - acc: 0.9126
Epoch 9/10
370379/370379 [==============================] - 239s 645us/step - loss: 0.2756 - acc: 0.9140
Epoc